In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [2]:
class WeakLearner:
    def __init__(self, model, i):
        self.__name = f"f{i+1}"
        self.__model = model
    
    def __sign(self, val):
        return 1 if val > 0 else -1
    
    def name(self):
        return self.__name
    
    def model(self):
        return self.__model
    
    def miss_classify(self, data, eval_data):
        self.__miss_data = []
        for i in range(len(data)):
            if self.__sign(self.model.predict(data)) != self.__sign(eval_data[i]):
                self.__miss_data.append(i)
        
    def calc_error_rate(self, w):
        self.__error_rate = np.sum(w[self.__miss_data])
    
    def calc_voting_power(self):
        self.__alpha = 1/2*np.log((1-self.__error_rate)/self.__error_rate)
        

In [4]:
def ShallowTree():
    return DecisionTreeClassifier(max_depth=2)

In [3]:
def classify(data, classification):
    return [1 if d.index(max(d)) == classification else -1]